In [25]:
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
import numpy as np
from typing import List, Dict, Union, Optional
from datetime import datetime
from pathlib import Path
import logging
import time
from ultralytics import YOLO
import json
import os

Пайплайн содержит три модели YOLOv8-cls для решения задачи классицикации (количесво классов num_classes= 1000).\
В функцию списком подаются 3 модели : yolov8n-cls, yolov8s-cls, yolov8m-cls

In [26]:
#Определение функции
def train_simple_pipeline(
    model_type: str,
    train_dir: str,
    val_dir: str,
    num_classes: int = 1000,
    epochs: int = 50
) -> list:

    
    valid_models = ["yolov8n-cls", "yolov8s-cls", "yolov8m-cls"]
    if model_type not in valid_models:
        raise ValueError(f"Выберите модель из: {valid_models}")

    #вызов модели
    model = YOLO(f"{model_type}.pt")

    #Обучение
    start_time = time.time()
    results = model.train(
        data="/home/c5/Рабочий стол/С[5]/C5_M2/images/Gauge_big/data.yaml",  # Требуется файл с путями
        epochs=epochs,
        imgsz=224,
        verbose=False
    )

    # Расчет метрик accuracy, roc_auc_score, f1_score (macro)
    val_results = model.predict(val_dir)
    
    true_labels = np.array([int(Path(img).parent.name) for img in val_results.imgs])
    pred_probs = np.array([pred.probs for pred in val_results])

    pred_labels = np.argmax(pred_probs, axis=1)
    metrics = {
        "accuracy": accuracy_score(true_labels, pred_labels),
        "roc_auc": roc_auc_score(true_labels, pred_probs, multi_class='ovr'),
        "f1_score": f1_score(true_labels, pred_labels, average='macro')
    }

    # Результаты
    output = {
        "model": model_type,
        "metrics": metrics,
        "time_sec": time.time() - start_time
    }
    
    with open(f"results_{model_type}.json", "w") as f:
        json.dump(output, f)

    return [model, metrics, output["time_sec"]]

##c


datasets/

├── train/

│   ├── class_0/

│   ├── class_1/

│   └── ... 

├── val/

│   ├── class_0/

│   ├── class_1/

│   └── ...

└── data.yaml

In [16]:
result = train_simple_pipeline(
    model_type="yolov8n-cls",
    train_dir="datasets/train",
    val_dir="datasets/val",
    num_classes=10,
    epochs=30
)

model, metrics, time = result
print(f"""
Результаты для {model_type}:
- Accuracy: {metrics['accuracy']:.2%}
- ROC-AUC:  {metrics['roc_auc']:.3f}
- F1-Score:  {metrics['f1_score']:.3f}
- Время обучения: {time:.1f} сек
""")

100%|█████████████████████████████████████████████████████████████████| 6.25M/6.25M [00:00<00:00, 10.3MB/s]


In [17]:
model.info()

YOLOv8n summary: 225 layers, 3,157,200 parameters, 0 gradients, 8.9 GFLOPs


(225, 3157200, 0, 8.8575488)